In [ ]:
!pip install --upgrade gupload
from pydrive.auth import GoogleAuth
from google.colab import auth
import os
import requests
import pandas as pd
import numpy as np
import time
import math
import datetime
from timeit import default_timer as timer
from bs4 import BeautifulSoup
from google.colab import drive
auth.authenticate_user()

drive.mount('/content/gdrive')
df = pd.read_csv('/content/gdrive/My Drive/PST_IL_movement_1992-2020.csv')
df['Date']=pd.to_datetime(df['Date'])
end_date = df['Date']<'2013-07-01'
df = df[end_date]
def determine_affected_player(row):
  if type(row['Acquired'])==str:
    return row['Acquired']
  if type(row['Relinquished'])==str:
    return row['Relinquished']
df['Affected Player']=df.apply(lambda row:determine_affected_player(row), axis=1)
start_season = 1992
end_year_season = 93
seasons_yr_start =[]
seasons_yr_end =[]
for i in range(21):
  seasons_yr_start.append(str(start_season+i))
  if not (i>6):
    seasons_yr_end.append(str(end_year_season+i))
for z in range(14):
  if z<10:
    seasons_yr_end.append('0'+str(z))
  else:
    seasons_yr_end.append(str(z))
season_names=[]
for r in range(len(seasons_yr_start)):
  season_names.append(seasons_yr_start[r]+'-' + seasons_yr_end[r])
print(season_names)
season_dates = {}
start_date = '{}-07-01'
end_date = '{}-06-30'
for szn in season_names:
  start_year = int(szn[0:4])
  end_year = start_year+1
  season_dates[szn]=[start_date.format(start_year),end_date.format(end_year)]
print(season_dates)

#Creating a function to determine exactly which season each injury took place in

def determine_szn(row):
  key_list = list(season_dates.keys())
  val_list = list(season_dates.values())
  for key,value in season_dates.items():
    strtdate = datetime.datetime.strptime(value[0], '%Y-%m-%d')
    enddate = datetime.datetime.strptime(value[1], '%Y-%m-%d')
    if ((row['Date']>=strtdate) and (row['Date']<=enddate)):
      pos = val_list.index(value)
      szn = key_list[pos]
      return szn
df['Season']=df.apply(lambda row:determine_szn(row), axis=1)
season_dates['2013-14'] = ['2013-07-01']
print('part 1 done')

#function to clean the 'Notes' column 

def clean_notes_col_pt_1(row):
  s = row['Notes']
  if '(' in s.lower():
    if s.count('(')==1:
      pt1 = s.index('(')
      pt2 = s.index(')')
      chk1=s[pt1:(pt2+1)]
      if 'out' in chk1:
        return s
      else:
        s = s.replace(chk1,'')
        return s
    if s.count('(')>1:
      pt1 = s.index('(')
      pt2 = s.index(')')
      chk1=s[pt1:(pt2+1)]
      chk1s = chk1.lower()
      pt3 = s.rindex('(')
      pt4 = s.rindex(')')
      nextchk1=s[pt3:(pt4+1)]
      nextchk1s=nextchk1.lower()
      nextchk1
      if ('out' in chk1s) and (not ('out' in nextchk1s)):
        s=s.replace(nextchk1,'')
        return s
      if (not ('out' in chk1s)) and ('out' in nextchk1s):
        s=s.replace(chk1,'')
        return s
      if (not ('out' in chk1s)) and (not ('out' in nextchk1s)):
        s = s.replace(nextchk1,'')
        s = s.replace(chk1,'')
        return s
      if ('out' in chk1s) and ('out' in nextchk1s):
        print(4)
        s = s.replace(nextchk1,'')
        return s
  else:
    return s
df['Notes']=df.apply(lambda row:clean_notes_col_pt_1(row), axis=1)

copied_df = df.copy()
!pip install nba_api
#cleaning the names of the nba players
from nba_api.stats.static import players
from nba_api.stats.static import teams 
from nba_api.stats.endpoints import boxscoreplayertrackv2
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
from datetime import datetime,timedelta
import time

acqcl = copied_df.columns.get_loc('Acquired')
rcl = copied_df.columns.get_loc('Relinquished')
acl = copied_df.columns.get_loc('Affected Player')
player_dict = players.get_players()
lst_of_player_names = []
names_not_in_nba_api = []
for key in player_dict:
  lst_of_player_names.append((key['full_name']))
rng = copied_df.shape[0]

#function to change the names of each player such that 
#their name on the data corresponds to the name that they 
#are listed under on the nba.com website
for i in range(rng):
  row = copied_df.iloc[i,:]
  player_name = row.loc['Affected Player']
  relinquished_name = row.loc['Relinquished']
  acquired_name = row.loc['Acquired']
  if player_name == acquired_name:
    for nba_name in lst_of_player_names:
      if nba_name in player_name:
        copied_df.iat[i,acqcl] = nba_name
        copied_df.iat[i,acl] = nba_name
  else:
    for nba_name in lst_of_player_names:
      if nba_name in player_name:
        copied_df.iat[i,rcl] = nba_name
        copied_df.iat[i,acl] = nba_name


print('part 3 done')

     |████████████████████████████████| 61kB 8.2MB/s 
     |████████████████████████████████| 81kB 10.4MB/s 
ERROR: earthengine-api 0.1.269 has requirement google-api-python-client<2,>=1.12.1, but you'll have google-api-python-client 1.7.10 which is incompatible.
  Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
  Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
Mounted at /content/gdrive
['1992-93', '1993-94', '1994-95', '1995-96', '1996-97', '1997-98', '1998-99', '1999-00', '2000-01', '2001-02', '2002-03', '2003-04', '2004-05', '2005-06', '2006-07', '2007-08', '2008-09', '2009-10', '2010-11', '2011-12', '2012-13']
{'1992-93': ['1992-07-01', '1993-06-30'], '1993-94': ['1993-07-01', '1994-06-30'], '1994-95': ['1994-07-01', '1995-06-30'], '1995-96': ['1995-07-01', '1996-06-30'], '1996-97': ['1996-07

In [ ]:
rng = df.shape[0]
#function to determine exactly
#what type of injury each player has incurred

def type_of_injury(row):
  notes = row['Notes']
  Injured_player = row['Relinquished']
  if type(Injured_player)==str:
    if (('IR' in notes) and ('with' in notes)):
      y = notes[:18]
      notes.replace(y,'')
      if '(' in notes:
        pos1 = notes.index('(')
        r = notes[pos1:]
        notes = notes.replace(r,'')
        if '&' in notes:
          pos2 = notes.index('&')
          u = notes[pos2] 
          notes = notes.replace(u,'')
          return notes
        elif '/' in notes:
          pos3=notes.index('/')
          z = notes[pos3:]
          notes = notes.replace(z,'')
          return notes
        else:
          return notes
      elif '&' in notes:
        pos2 = notes.index('&')
        u = notes[pos2] 
        notes = notes.replace(u,'')
        return notes
      elif '/' in notes:
        pos3=notes.index('/')
        z = notes[pos3:]
        notes = notes.replace(z,'')
        return notes
      else:
        return notes
    elif (('IL' in notes) and ('with' in notes)):
      y = notes[:18]
      notes = notes.replace(y,'')
      if '(' in notes:
        pos1 = notes.index('(')
        r = notes[pos1:]
        notes = notes.replace(r,'')
        if '&' in notes:
          pos2 = notes.index('&')
          u = notes[pos2:] 
          notes = notes.replace(u,'')
          return notes
        elif '/' in notes:
          pos3=notes.index('/')
          z = notes[pos3:]
          notes = notes.replace(z,'')
          return notes
        else:
          return notes
      elif '&' in notes:
        pos2 = notes.index('&')
        u = notes[pos2:] 
        notes = notes.replace(u,'')
        return notes
      elif '/' in notes:
        pos3=notes.index('/')
        z = notes[pos3:]
        notes = notes.replace(z,'')
        return notes
      else:
        return notes
    else:
      rep = notes[:11]
      notes = notes.replace(rep,'')
      return notes
df['Injury Type']=df.apply(lambda row:type_of_injury(row), axis=1)



part 4 done


In [ ]:
rng = df.shape[0]
dict_of_incomplete_players = {}
indicies_to_drop = []
season_names.append('2013-14')

#deleting consecutive instances of relinquished players (when player is not out for the season)
#and next relinquished occurs 
for i in range(rng):
  row = df.iloc[i,:]
  player_name = row.loc['Affected Player']
  relinquished_name = row.loc['Relinquished']
  notes_first = row.loc['Notes']
  season_first = row.loc['Season']
  impindex = season_names.index(season_first)
  season_next = season_names[impindex+1] 
  if type(relinquished_name)==str:
    for r in range(i+1,rng):
      list_to_append = [i,r]
      next_row = df.iloc[r,:]
      notes_second = row.loc['Notes']
      acquired_name = next_row.loc['Acquired']
      season_second = next_row.loc['Season']
      next_relinquished_name = next_row.loc['Relinquished']
      next_player_name=next_row.loc['Affected Player']
      if (next_player_name == player_name):
        if (acquired_name==player_name):
          break
        else:
          if (('out for season' in notes_first) or ('out for season' in notes_second)):
            if season_second == season_next:
              if player_name in dict_of_incomplete_players:
                dict_of_incomplete_players[player_name].append(list_to_append)
                break
              else:
                dict_of_incomplete_players[player_name]=[list_to_append]
                break
            else:
              break              
          else:
            indicies_to_drop.append(i)
            break
print(indicies_to_drop)
# print(dict_of_incomplete_players)
JUST_IN_CASE = df.copy()

df = df.reset_index(drop=True)
df = df.drop(labels=indicies_to_drop,axis=0)
df = df.reset_index(drop=True)
copied_df = df.copy()




[8, 12, 17, 18, 36, 41, 50, 56, 74, 78, 94, 95, 100, 107, 108, 111, 161, 191, 229, 292, 321, 332, 459, 486, 510, 562, 663, 664, 667, 688, 697, 759, 909, 999, 1060, 1113, 1219, 1224, 1252, 1290, 1318, 1347, 1379, 1413, 1437, 1485, 1492, 1506, 1531, 1710, 1738, 1740, 1748, 1767, 1781, 1789, 1801, 1889, 1910, 1940, 1950, 1952, 2053, 2102, 2147, 2223, 2224, 2228, 2235, 2262, 2433, 2444, 2445, 2512, 2533, 2547, 2556, 2615, 2632, 2633, 2634, 2635, 2636, 2637, 2638, 2639, 2640, 2641, 2642, 2643, 2657, 2659, 2685, 2735, 2737, 2746, 2777, 2794, 2833, 2864, 2913, 2922, 2966, 3038, 3041, 3042, 3052, 3057, 3086, 3109, 3112, 3116, 3120, 3125, 3127, 3129, 3139, 3146, 3151, 3156, 3159, 3161, 3171, 3181, 3182, 3195, 3196, 3199, 3201, 3241, 3260, 3270, 3312, 3325, 3387, 3421, 3431, 3536, 3538, 3586, 3600, 3619, 3645, 3654, 3691, 3722, 3733, 3896, 3928, 3942, 3961, 3967, 3972, 4096, 4114, 4149, 4182, 4231, 4256, 4280, 4284, 4327, 4335, 4337, 4363, 4399, 4401, 4403, 4411, 4413, 4433, 4449, 4456, 4462, 44

In [ ]:
#finding instances where next occurence
#of being acquired (after initially being acquired)
#occurs in the same season and deleting the latter occurence

rng = df.shape[0]
double_acquired = []
for i in range(rng):
  row = copied_df.iloc[i,:]
  notes = row.loc['Notes']
  player_name = row.loc['Affected Player']
  relinquished_name = row.loc['Relinquished']
  acquired_name = row.loc['Acquired']
  season_first = row.loc['Season']
  if player_name == acquired_name:
    for z in range(i+1,rng):
      next_row = copied_df.iloc[z,:]
      next_player_name = next_row.loc['Affected Player']
      next_relinquished_name = next_row.loc['Relinquished']
      next_acquired_name = next_row.loc['Acquired']
      next_season = next_row.loc['Season']
      next_notes = next_row.loc['Notes']
      if season_first == next_season:
        if player_name == next_player_name:
          if player_name == next_acquired_name and (season_first==next_season):
            double_acquired.append(i)
            break
          else:
            break
      else:
        break


print('part 5 done')          


part 5 done


In [ ]:
new_dropping_list = sorted(double_acquired)
df=df.reset_index(drop=True)
df = df.drop(labels=new_dropping_list,axis=0) 
df=df.reset_index(drop=True)
copied_df = df.copy()



#finding instances where next occurence
#of being relinquished (after initially being relinquished)
#occurs in the same season
rng = copied_df.shape[0]
ghost_injury = []
names = {}
more_shit_to_drop_doubleacquired = []
for i in range(rng):
  row = copied_df.iloc[i,:]
  notes = row.loc['Notes']
  player_name = row.loc['Affected Player']
  relinquished_name = row.loc['Relinquished']
  season_first = row.loc['Season']
  if player_name == relinquished_name:
    for z in range(i+1,rng):
      next_row = copied_df.iloc[z,:]
      next_player_name = next_row.loc['Affected Player']
      next_relinquished_name = next_row.loc['Relinquished']
      next_acquired_name = next_row.loc['Acquired']
      next_season = next_row.loc['Season']
      next_notes = next_row.loc['Notes']
      if player_name == next_player_name:
        if player_name == next_acquired_name:
          break
        elif player_name == next_relinquished_name:
          if season_first == next_season:
            ghost_injury.append(i)
            if player_name in names:
              names[player_name].append([i,z])
              break
            else:
              names[player_name] = [[i,z]]
              break
            break
df = df.reset_index(drop=True)
copied_df = df.copy()


print('part 6 done')


part 6 done


In [ ]:

#newdict_out are players who were out for the 
#season and got another injury/update
#consider removing instances where they're given surgery
newdict_out = {}
#newdict_notout4szn are players who weren't considered out for either portion of the year
newdict_notout4szn = {}
consider_dropping = []
#others are players who were out for surgery at first
#but came back? go through separately

others = {}
#sorting through these double occurences of relinquished
#if they are relisted for being out for surgery
#remove the instance of surgery
#if they are listed out for the season for surgery FIRST
#and relisted on the list for some reason
#evaluate instances separetely 


for key,value in names.items():
  if len(value)==1:
    indx_one = value[0][0]
    indx_two = value[0][1]
    row_one = copied_df.iloc[indx_one,:]
    row_two = copied_df.iloc[indx_two,:]
    notes_one = row_one.loc['Notes']
    notes_two = row_two.loc['Notes']
    if ('out' in notes_one) or ('out' in notes_two):
      if 'surgery' in notes_two:
        consider_dropping.append(indx_two)
      elif 'surgery' in notes_one:
        others[key]=value
      elif key in newdict_out:
        newdict_out[key].append(value)
      else:
        newdict_out[key]=value
    else:
      if key in newdict_notout4szn:
        newdict_notout4szn[key].append(value)
      else:
        newdict_notout4szn[key]=value
  else:
    for val in value:
      indx_one = val[0]
      indx_two = val[1]
      row_one = copied_df.iloc[indx_one,:]
      row_two = copied_df.iloc[indx_two,:]
      notes_one = row_one.loc['Notes']
      notes_two = row_two.loc['Notes']
      if ('out' in notes_one) or ('out' in notes_two):
        if 'surgery' in notes_two:
          consider_dropping.append(indx_two)
        elif 'surgery' in notes_one:
          others[key]=val
        elif key in newdict_out:
          newdict_out[key].append(val)
        else:
          newdict_out[key]=val
      else:
        if key in newdict_notout4szn:
          newdict_notout4szn[key].append(val)
        else:
          newdict_notout4szn[key]=val

print(newdict_out)
print('above is newdict_out')
print(newdict_notout4szn)
print('above is newdict_notout4szn')
print(consider_dropping)
print('rows to consider dropping is above')
print(others)
print('above is the others dict')
df = df.drop(labels=ghost_injury,axis=0)
df=df.reset_index(drop=True)
copied_df = df.copy()
backup = df.copy()


{'Elliot Williams': [[16022, 16056]], 'Andray Blatche': [[16077, 16104]], 'Emeka Okafor': [[16117, 16139]]}
above is newdict_out
{}
above is newdict_notout4szn
[]
rows to consider dropping is above
{'Mitchell Butler': [[1237, 1387]], 'Jermareo Davidson': [[12072, 12141]]}
above is the others dict


In [ ]:
df = backup.copy()
#an injury categorizing function that 
#determines exactly WHICH body part got injured

def injury_categorizer(row):
  injury_type = row['Notes']
  relinquished_player = row['Relinquished']
  player_name = row.loc['Affected Player']
  if relinquished_player == player_name:
    injury_type = injury_type.lower()
    foot = ['toe','heel','metatarsal','ankle','achilles','achilled','foot','feet','plantar','metatasalgia']
    upper_leg = ['thigh','abductor','buttock','adductor','hamstring','groin','femur','leg','quadricep']
    knee=['acl','mcl','knee','meniscus','patella']
    lower_leg = ['tibia','shin','fibula','calf']
    arm = ['bicep','tricep','arm','elbow']
    hand = ['wrist','pinky','finger','thumb','hand']
    upper_body = ['clavicle','sternum','pectoral','chest','shoulder','rotator','delt']
    core_and_hip = ['abdom','adominal','oblique','rib','stomach muscle','hip']
    head_and_neck = ['facial','cervical','teeth','concussion','face','orbit','tooth','jaw','nose','collarbone','eye','head','neck','cheekbone','dental']
    back = ['spine','herniated discs','herniated disc','back','vertebra','herniated','lumbro-sacral','lumbrosacral','lumbar','spinal','tail']

    mental_illness = ['depression','mental','clinical']
    illness = ['illness','virus','flu','sick','esophagitis','migraine','allergic','allergy','salmonella','nausea','pox','infection','syndrome','viral','stomach','sick','pneumonia','dizziness','poisoning','bronchitis','gastro','strep','adverse reaction','sinus']
    miscellaneous = ['personal','family','death','birth','tonsillitis']
    if any(i in injury_type for i in foot):
      return 'foot'
    elif any(i in injury_type for i in lower_leg):
      return 'lower leg'
    elif any(i in injury_type for i in knee):
      return 'knee'
    elif any(i in injury_type for i in upper_leg):
      return 'upper leg'
    elif any(i in injury_type for i in arm):
      return 'arm'
    elif any(i in injury_type for i in hand):
      return 'hand'
    elif any(i in injury_type for i in upper_body):
      return 'upper body'
    elif any(i in injury_type for i in core_and_hip):
      return 'core and hip'
    elif any(i in injury_type for i in head_and_neck):
      return 'head and neck'
    elif any(i in injury_type for i in mental_illness):
      return 'mental illness'
    elif any(i in injury_type for i in back):
      return 'back'
    elif any(i in injury_type for i in illness):
      return 'illness'
    elif any(i in injury_type for i in miscellaneous):
      return 'miscellaneous'
    else:
      return 'other'
  else:
    return 'Acquired'
df['Injury Part']=df.apply(lambda x:injury_categorizer(x), axis=1)
copied_df=df.copy()
backup = df.copy()


In [ ]:
checkpoint_dates = {}
beginning_dates_lst = ['1992-11-09', '1993-11-08', '1994-11-07', '1995-11-06', '1996-11-04', '1997-11-03', '1999-02-07', '1999-11-05', '2000-11-03', '2001-11-02', '2002-11-02', '2003-11-01', '2004-11-05', '2005-11-04', '2006-11-03', '2007-11-03', '2008-11-01', '2009-11-01', '2010-11-01', '2011-12-28', '2012-11-03','2013-10-01']
for i in range(len(season_names)):
  checkpoint_dates[season_names[i]]=beginning_dates_lst[i]
seasons_yr_start.append(2013)
pseudoreturn = []
pseudoreturn_dates = {}
b='{}-10-01'
for year in seasons_yr_start:
  pseudoreturn.append(b.format(year))
for i in range(len(beginning_dates_lst)):
  pseudoreturn_dates[season_names[i]] = pseudoreturn[i]
print(pseudoreturn_dates)
print(checkpoint_dates)
JUST_IN_CASE_FOUR = df.copy()

#finding players who were never 'returned' to the starting lineup after their out for season injuries
#adding in a generic row to establish a return date to get approximate injury length
#While this is a strong assumption
#it is a viable solution considering we do not have the exact data of when a player has 'fully healed'
#from a particular injury

#also, for players that were out for season and relisted 
#at the beginning of the next season as out (for an injury to the same body part)
#(for presumambly the same injury)
#delete the 'inital' row that they were taking up

copied_df = df.copy()
to_drop = []

rng = copied_df.shape[0]
for i in range(rng):
  row = copied_df.iloc[i,:]
  notes = row.loc['Notes']
  player_name = row.loc['Affected Player']
  relinquished_name = row.loc['Relinquished']
  acquired_name = row.loc['Acquired']
  season_first = row.loc['Season']
  season_first_int = int(season_first[:4])
  impindex = season_names.index(season_first)
  season_next = season_names[impindex+1]
  acquired_note = 'activated from IL'
  inj_part = row.loc['Injury Part']
  if ((player_name == relinquished_name) and ('out for season' in notes)):
    if season_first == '2012-13':
      season_next = '2013-14'
      date = pseudoreturn_dates[season_next]
      Team = row.loc['Team']
      row2add = {'Date':date,'Team':Team,'Acquired':player_name,'Relinquished':acquired_name,'Notes':acquired_note,'Affected Player':player_name,'Season':season_next,'Injury Type':'','Injury Part':'Acquired'}
      copied_df=copied_df.append(row2add,ignore_index=True)
    else:
      for z in range(i+1,rng):
        next_row = copied_df.iloc[z,:]
        next_player_name = next_row.loc['Affected Player']
        next_relinquished_name = next_row.loc['Relinquished']
        next_acquired_name = next_row.loc['Acquired']
        season_second = next_row.loc['Season']
        next_notes = next_row.loc['Notes']
        second_date = next_row.loc['Date']
        Team = next_row.loc['Team']
        next_injury_part = next_row.loc['Injury Part']
        season_second_int = int(season_second[:4])
        if player_name == next_player_name:
          if player_name == next_acquired_name:
            if (season_second_int-season_first_int)<3:
              break
            else:
              date_fill = pseudoreturn_dates[season_next]
              row2add = {'Date':date_fill,'Team':Team,'Acquired':player_name,'Relinquished':acquired_name,'Notes':acquired_note,'Affected Player':player_name,'Season':season_next,'Injury Type':'','Injury Part':'Acquired'}
              df=df.append(row2add,ignore_index=True)
              break
          else:
            if season_second == season_next:
              comp_date = datetime.strptime(checkpoint_dates[season_second], '%Y-%m-%d')
              if (second_date < comp_date):
                if inj_part == next_injury_part:
                  if ('out for season' in next_notes):
                    season_next = season_names[impindex+2]
                    date = pseudoreturn_dates[season_next]
                    row2add = {'Date':date,'Team':Team,'Acquired':player_name,'Relinquished':acquired_name,'Notes':acquired_note,'Affected Player':player_name,'Season':season_next,'Injury Type':'','Injury Part':'Acquired'}
                    copied_df=copied_df.append(row2add,ignore_index=True)
                    to_drop.append(z)
                  else:
                    to_drop.append(z)
                  break
                else:
                  season_next = season_names[(impindex+1)]
                  date = season_dates[season_next][0]
                  row2add = {'Date':date,'Team':Team,'Acquired':player_name,'Relinquished':acquired_name,'Notes':acquired_note,'Affected Player':player_name,'Season':season_second,'Injury Type':'','Injury Part':'Acquired'}
                  df=df.append(row2add,ignore_index=True)
                  break
              else:
                date = pseudoreturn_dates[season_second]
                row2add = {'Date':date,'Team':Team,'Acquired':player_name,'Relinquished':acquired_name,'Notes':acquired_note,'Affected Player':player_name,'Season':season_second,'Injury Type':'','Injury Part':'Acquired'}
                copied_df=copied_df.append(row2add,ignore_index=True)
                break
            else:
              date = pseudoreturn_dates[season_second]
              row2add = {'Date':date,'Team':Team,'Acquired':player_name,'Relinquished':acquired_name,'Notes':acquired_note,'Affected Player':player_name,'Season':season_second,'Injury Type':'','Injury Part':'Acquired'}
              copied_df=copied_df.append(row2add,ignore_index=True)
              break
print(to_drop)
copied_df = copied_df.reset_index(drop=True)
copied_df = copied_df.drop(labels=to_drop,axis=0)
copied_df = copied_df.reset_index(drop=True)
copied_df['Date']=pd.to_datetime(copied_df['Date'])
copied_df = copied_df.sort_values(by='Date')
copied_df = copied_df.reset_index(drop=True)
df = copied_df.copy()
backup = copied_df.copy()


{'1992-93': '1992-10-01', '1993-94': '1993-10-01', '1994-95': '1994-10-01', '1995-96': '1995-10-01', '1996-97': '1996-10-01', '1997-98': '1997-10-01', '1998-99': '1998-10-01', '1999-00': '1999-10-01', '2000-01': '2000-10-01', '2001-02': '2001-10-01', '2002-03': '2002-10-01', '2003-04': '2003-10-01', '2004-05': '2004-10-01', '2005-06': '2005-10-01', '2006-07': '2006-10-01', '2007-08': '2007-10-01', '2008-09': '2008-10-01', '2009-10': '2009-10-01', '2010-11': '2010-10-01', '2011-12': '2011-10-01', '2012-13': '2012-10-01', '2013-14': '2013-10-01'}
{'1992-93': '1992-11-09', '1993-94': '1993-11-08', '1994-95': '1994-11-07', '1995-96': '1995-11-06', '1996-97': '1996-11-04', '1997-98': '1997-11-03', '1998-99': '1999-02-07', '1999-00': '1999-11-05', '2000-01': '2000-11-03', '2001-02': '2001-11-02', '2002-03': '2002-11-02', '2003-04': '2003-11-01', '2004-05': '2004-11-05', '2005-06': '2005-11-04', '2006-07': '2006-11-03', '2007-08': '2007-11-03', '2008-09': '2008-11-01', '2009-10': '2009-11-01'

In [ ]:
df = copied_df.copy()
rng = copied_df.shape[0]
double_relinquished = []
#double checking to see if there are any instances of a player
#being relinquished two times in a row without being acquired
for i in range(rng):
  row = copied_df.iloc[i,:]
  player_name = row.loc['Affected Player']
  relinquished_name = row.loc['Relinquished']
  acquired_name = row.loc['Acquired']
  if player_name == relinquished_name:
    for z in range(i+1,rng):
      next_row = copied_df.iloc[z,:]
      next_player_name = next_row.loc['Affected Player']
      next_relinquished_name = next_row.loc['Relinquished']
      next_acquired_name = next_row.loc['Acquired']
      if next_acquired_name==player_name:
        break
      elif next_relinquished_name==player_name:
        double_relinquished.append([i,z])
        break
      else:
        continue


drop_ici = []
copied_df = df.copy()
for i in range(len(double_relinquished)):
  drop_ici.append(double_relinquished[i][0])
copied_df = copied_df.reset_index(drop=True)
copied_df = copied_df.drop(labels=drop_ici,axis=0)
copied_df = copied_df.reset_index(drop=True)
copied_df['Date']=pd.to_datetime(copied_df['Date'])
# backup = copied_df.copy()
copied_df = backup.copy()
df = backup.copy()

#after adding in the generic rows
#we are ready to create a function and a column
#to determine the exact duration of a particular injury

def determine_injury_length(row):
  rng = copied_df.shape[0]
  notes = row.loc['Notes']
  player_name = row.loc['Affected Player']
  relinquished_name = row.loc['Relinquished']
  acquired_name = row.loc['Acquired']
  season_first = row.loc['Season']
  date = row.loc['Date']
  row_loc = copied_df.index[((copied_df['Date']==date) & (copied_df['Affected Player'] == player_name))][0]
  if player_name == relinquished_name:
    for z in range(row_loc+1,rng):
      next_row = copied_df.iloc[z,:]
      next_player_name = next_row.loc['Affected Player']
      next_relinquished_name = next_row.loc['Relinquished']
      next_acquired_name = next_row.loc['Acquired']
      next_date = next_row.loc['Date']
      if (player_name == next_player_name):
        if (player_name == next_acquired_name):
          return (next_date - date).days
        else:
           return 'ERROR'
      elif (z == (rng-1)):
        return 'Never Acquired Again'
      else:
        continue
  else:
    return 'Acquired'
copied_df['Injury Duration']=df.apply(lambda row:determine_injury_length(row), axis=1)

copied_df.to_csv('cleaned_names_1.csv')
!gupload --to '12W7wtP_PL3oV5osKCwxM2cQWxzu1yETv' cleaned_names_1.csv
print('Finished')

Uploading file: cleaned_names_1.csv
Finished


In [ ]:
#deleting others and miscellaneous here
#as well as the 'never acquired again' instances
#i.e. the instances that won't be very useful to our data

copied_df = copied_df.reset_index(drop=True)
df = copied_df.copy()
rng = copied_df.shape[0]
to_drop = []
for i in range(rng):
  row = copied_df.iloc[i,:]
  player_name = row.loc['Affected Player']
  relinquished_name = row.loc['Relinquished']
  acquired_name = row.loc['Acquired']
  injury_part = row.loc['Injury Part']
  injury_duration = row.loc['Injury Duration']
  mask1= (injury_part == 'other')
  mask2 = (injury_part == 'miscellaneous')
  mask3 = (injury_duration == 'Never Acquired Again')
  mask4 = (injury_duration == 'Acquired')
  mask5 = (injury_part == 'mental illness')
  mask6 = (injury_part == 'illness')
  mask8 = (injury_duration == 'ERROR')
  masklst = [mask1,mask2,mask3,mask4,mask5,mask6,mask8]
  if any(masklst):
    to_drop.append(i)
  else:
    continue
print(to_drop)
copied_df = copied_df.reset_index(drop=True)
copied_df = copied_df.drop(labels=to_drop,axis=0)
copied_df = copied_df.reset_index(drop=True)

df = copied_df.copy()
rng = copied_df.shape[0]
to_drop = []
#also deleting instances where the injury is over 725 days
#as that can skew our data 
for i in range(rng):
  row = copied_df.iloc[i,:]
  player_name = row.loc['Affected Player']
  relinquished_name = row.loc['Relinquished']
  acquired_name = row.loc['Acquired']
  injury_part = row.loc['Injury Part']
  injury_duration = row.loc['Injury Duration']
  mask7 = (injury_duration > 725)
  if mask7:
    to_drop.append(i)
copied_df = copied_df.reset_index(drop=True)
copied_df = copied_df.drop(labels=to_drop,axis=0)
copied_df = copied_df.reset_index(drop=True)
df = copied_df.copy()
backup2=df.copy()
copied_df.to_csv('cleaned_names_1.csv')
!gupload --to '12W7wtP_PL3oV5osKCwxM2cQWxzu1yETv' cleaned_names_1.csv
print('Finished')

print('Done latest iteration')

#making a new DF where it has a list of players and the duration they've been out for various injuries
#as well as showing exactly how many injuries they incurred to each body part
rng = copied_df.shape[0]
player_history_dict = {}
for i in range(rng):
  row = copied_df.iloc[i,:]
  player_name = row.loc['Affected Player']
  injury_part = row.loc['Injury Part']
  injury_duration = row.loc['Injury Duration']
  if player_name in player_history_dict:
    player_history_dict[player_name][injury_part] += injury_duration
    player_history_dict[player_name][injury_part+'_count'] += 1
  else:
    row2add = {'foot':0,'knee':0,'upper leg':0,'lower leg':0,'back':0,'arm':0,'hand':0,'upper body':0,'core and hip':0,'head and neck':0,'foot_count':0,'knee_count':0,'upper leg_count':0,'lower leg_count':0,'back_count':0,'arm_count':0,'hand_count':0,'upper body_count':0,'core and hip_count':0,'head and neck_count':0}
    row2add[injury_part] += injury_duration
    row2add[injury_part+'_count'] += 1
    player_history_dict[player_name]=row2add

#the player injury history is the data we will use to help our analysis
#with injuries post 2013 
#a player's injury history may have a great impact on the severity of their future injuries
player_history = pd.DataFrame.from_dict(player_history_dict, orient='index')
player_history.to_csv('FINAL_player_injury_history.csv')
!gupload --to '12W7wtP_PL3oV5osKCwxM2cQWxzu1yETv' FINAL_player_injury_history.csv
print('Finished')
print('Done latest iteration')

[]
Uploading file: cleaned_names_1.csv
Finished
Done latest iteration
Uploading file: FINAL_player_injury_history.csv
Finished
Done latest iteration
